In [11]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

import subprocess
from datetime import datetime
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter

import numpy as np

log_file = "notifications.log"

def send_notification(title, message):
    subprocess.run(["notify-send", title, message])
    with open(log_file, "a") as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {title}: {message}\n")

send_notification("Sanskrit Training","Sanskrit Training Started")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
send_notification("Sanskrit Training","Sanskrit model training started...")
# Hyperparameters
batch_size = 64
block_size = 128
max_iters = 1500
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

send_notification("Sanskrit Training", device)

with open('/home/anon/dataset/Sankrit_Corpus/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

str_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_str = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: [int_to_str[i] for i in l]

# Memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "/home/anon/dataset/Sankrit_Corpus/train_split.txt" if split == 'train' else "/home/anon/dataset/Sankrit_Corpus/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0, file_size - block_size * batch_size)
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

def get_batch(split, batch_size):
    data = get_random_chunk(split)
    data_len = len(data)
    if data_len <= block_size + 1:
        return None, None  # Not enough data for a batch
    batch_size = min(batch_size, (data_len - 1) // block_size)
    ix = torch.randint(data_len - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    valid_mask = (y < vocab_size).all(dim=1)
    x = x[valid_mask]
    y = y[valid_mask]
    if len(x) == 0 or len(y) == 0:
        return None, None  # No valid batches
    x, y = x.to(device), y.to(device)
    return x, y

def bleu_score(reference, candidate, k=4):
    def n_gram_precision(ref, cand, n):
        ref_ngrams = Counter([tuple(ref[i:i+n]) for i in range(len(ref)-n+1)])
        cand_ngrams = Counter([tuple(cand[i:i+n]) for i in range(len(cand)-n+1)])
        numerator = sum((cand_ngrams & ref_ngrams).values())
        denominator = max(1, sum(cand_ngrams.values()))
        return numerator / denominator

    precisions = [n_gram_precision(reference, candidate, i) for i in range(1, k+1)]
    bleu = np.exp(sum(np.log(p) if p > 0 else 0 for p in precisions) / k)
    bp = np.exp(1 - len(reference) / len(candidate)) if len(candidate) < len(reference) else 1.0
    return bleu * bp

@torch.no_grad()
def estimate_loss_and_metrics(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        all_preds = []
        all_targets = []
        bleu_scores = []
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size)
            if X is None:
                continue  # Skip this iteration if not enough data for a batch
            X = X.view(-1, X.size(-1))
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            targets = Y.cpu().numpy()
            preds_flat = preds.flatten()
            targets_flat = targets.flatten()
            all_preds.extend(preds_flat)
            all_targets.extend(targets_flat)
            preds_list = [np.atleast_1d(p).tolist() for p in preds_flat]
            targets_list = [np.atleast_1d(t).tolist() for t in targets_flat]
            for p, t in zip(preds_list, targets_list):
                p_sentence = decode(p)
                t_sentence = decode(t)
                bleu_scores.append(bleu_score(t_sentence, p_sentence))
        cm = confusion_matrix(all_targets, all_preds)
        tp = np.diag(cm).sum()
        fp = cm.sum(axis=0) - np.diag(cm)
        fn = cm.sum(axis=1) - np.diag(cm)
        tn = cm.sum() - (fp + fn + tp)
        out[split] = {
            'loss': losses.mean().item(),
            'accuracy': accuracy_score(all_targets, all_preds),
            'precision': precision_score(all_targets, all_preds, average='macro', zero_division=0),
            'recall': recall_score(all_targets, all_preds, average='macro', zero_division=0),
            'f1': f1_score(all_targets, all_preds, average='macro', zero_division=0),
            'bleu': sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0.0,
            'true_positive': tp,
            'true_negative': tn.sum(),
            'false_positive': fp.sum(),
            'false_negative': fn.sum()
        }
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        with torch.no_grad():
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = GPTLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
if os.path.exists('/home/anon/training/model_sans.pkl'):
    send_notification("Sanskrit Training",'loading model parameters...')
    with open('/home/anon/training/model_sans.pkl', 'rb') as f:
        model = pickle.load(f)
    send_notification("Sanskrit Training",'loaded successfully!')
m = model.to(device)

start_time = time.time()

for iter in range(max_iters):
    xb, yb = get_batch('train', batch_size)
    if xb is None:
        continue  # Skip this iteration if not enough data for a batch
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % eval_iters == 0:
        losses = estimate_loss_and_metrics(model)
        end_time = time.time()
        send_notification("Sanskrit Training", f"step: {iter}, train loss: {losses['train']['loss']:.3f}, val loss: {losses['val']['loss']:.3f}, "+
                                                f"train accuracy: {losses['train']['accuracy']:.3f}, val accuracy: {losses['val']['accuracy']:.3f}, "+
                                                f"train precision: {losses['train']['precision']:.3f}, val precision: {losses['val']['precision']:.3f}, "+
                                                f"train recall: {losses['train']['recall']:.3f}, val recall: {losses['val']['recall']:.3f}, "+
                                                f"train f1: {losses['train']['f1']:.3f}, val f1: {losses['val']['f1']:.3f}, "+
                                                f"train BLEU: {losses['train']['bleu']:.3f}, val BLEU: {losses['val']['bleu']:.3f}, "+
                                                f"train TP: {losses['train']['true_positive']}, val TP: {losses['val']['true_positive']}, "+
                                                f"train TN: {losses['train']['true_negative']}, val TN: {losses['val']['true_negative']}, "+
                                                f"train FP: {losses['train']['false_positive']}, val FP: {losses['val']['false_positive']}, "+
                                                f"train FN: {losses['train']['false_negative']}, val FN: {losses['val']['false_negative']} "+
                                                f"in {(end_time - start_time):.2f} seconds.")


print("Training complete.")
with open('/home/anon/training/model_sans.pkl', 'wb') as f:
    pickle.dump(model, f)
send_notification("Sanskrit Training",'Sanskrit model saved')


Training complete.
